In [1]:
using EtherArrays
using StaticArrays
using KernelAbstractions
using oneAPI

In [2]:
# struct Par{N, T}
#     mass::E2Scalar{T}
#     rho::E2Scalar{T}
#     pos::E2Vector{N, T}
#     vel::E2Vector{N, T}
#     acc::E2Vector{N, T}
#     stress::E2SquareMatrix{N, T}
# end
struct Par{N, T}
    mass::StaticScalar{T}
    rho::StaticScalar{T}
    pos::StaticVector{N, T}
    vel::StaticVector{N, T}
    acc::StaticVector{N, T}
    stress::StaticMatrix{N, N, T}
end

In [3]:
const kBackend = oneAPI.oneAPIBackend()
const Tcontainer = oneAPI.oneArray
const Tf = Float32

Float32

In [4]:
const nt = (
    mass = 1,
    rho = 2,
    pos = 3,
    vel = 5,
    acc = 7,
    stress = 9
)
data_cpu = randn(Tf, 3, 12)
data_gpu = Tcontainer(data_cpu)

3×12 oneArray{Float32, 2, oneAPI.oneL0.DeviceBuffer}:
 0.289895   0.744079   0.0424512  …  -1.08298     1.01485  -1.80636
 0.111098  -0.209039   0.546642      -0.0336454   1.11407   1.04032
 1.84304   -0.880107  -0.826937       0.472491   -1.91381   0.372863

In [5]:
@kernel function par_ker!(data)
    idx::Int = @index(Global)
    mass = E2Scalar(idx, nt.mass, data)
    rho = E2Scalar(idx, nt.rho, data)
    pos = E2Vector{2}(idx, nt.pos, data)
    vel = E2Vector{2}(idx, nt.vel, data)
    acc = E2Vector{2}(idx, nt.acc, data)
    stress = E2SquareMatrix{2}(idx, nt.stress, data)
    vel .+= acc * 0.1f0
    par = Par{2, Tf}(mass, rho, pos, vel, acc, stress)
    par.pos .+= par.vel * 1.0f0
end

par_ker! (generic function with 4 methods)

In [6]:
par_ker!(kBackend, 3)(data_gpu, ndrange=(3,))
KernelAbstractions.synchronize(kBackend)

In [7]:
data_gpu

3×12 oneArray{Float32, 2, oneAPI.oneL0.DeviceBuffer}:
 0.289895   0.744079  0.854804   -4.29698  …  -1.08298     1.01485  -1.80636
 0.111098  -0.209039  0.313539    0.70643     -0.0336454   1.11407   1.04032
 1.84304   -0.880107  0.0624732  -1.33463      0.472491   -1.91381   0.372863

In [8]:
v = E2Vector{2}(1, 1, data_cpu)

2-element E2Vector{2, Float32} with indices SOneTo(2):
 0.2898947
 0.74407876

In [18]:
row!(v, 1)

1

In [20]:
v

2-element E2Vector{2, Float32} with indices SOneTo(2):
 0.2898947
 0.74407876

In [21]:
data_cpu

3×12 Matrix{Float32}:
 0.289895   0.744079   0.0424512  …  -1.08298     1.01485  -1.80636
 0.111098  -0.209039   0.546642      -0.0336454   1.11407   1.04032
 1.84304   -0.880107  -0.826937       0.472491   -1.91381   0.372863